In [1]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install 'temporal_fusion_transformer[training] @ git+https://github.com/aaarrti/temporal_fusion_transformer.git@develop'
!pip install "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

First, we'd need to upload dataset to GDrive

In [2]:
from google.colab import drive

drive.mount("/content/drive")
data_path = "/content/drive/MyDrive/datasets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import jax
from absl_extra import logging_utils
import temporal_fusion_transformer as tft

tf.config.set_visible_devices([], "GPU")
logging_utils.setup_logging(log_level="INFO")
jax.devices()

[gpu(id=0)]

In [4]:
config = tft.get_config("electricity")
config

fixed_params:
  input_known_categorical_idx:
  - 3
  - 4
  - 5
  - 6
  input_known_real_idx:
  - 1
  - 2
  input_observed_idx: []
  input_static_idx:
  - 0
  known_categories_sizes:
  - 31
  - 24
  - 7
  - 12
  num_encoder_steps: 168
  num_outputs: 1
  static_categories_sizes:
  - 369
  total_time_steps: 192
hyperparams:
  dropout_rate: 0.1
  latent_dim: 160
  num_attention_heads: 4
  num_decoder_blocks: 1
  quantiles:
  - 0.1
  - 0.5
  - 0.9
optimizer:
  clipnorm: 0.0
  decay_alpha: 0.05
  decay_steps: 0.0
  ema: 0.0
  learning_rate: 0.0005
prng_seed: 69
shuffle_buffer_size: 1024

Run training

In [ ]:
tft.training_scripts.train_experiment_on_single_device(
    data_dir=data_path,
    batch_size=32,
    experiment_name="electricity",
    mixed_precision=True,
    config=config,
    save_path="/content/drive/MyDrive/models",
    jit_module=True,
)

INFO:absl:Writing tensorboard logs to tensorboard/electricity/20230819-1055
